In [2]:
import numpy as np
"""
Computes camera matrix given image and real-world coordinates.

    Args:
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        camera_matrix: The calibrated camera matrix (3x4 matrix).
"""
"I guess such corresponding points are hand-crafted"
real_XY = np.load('real_XY.npy')
front_image = np.load('front_image.npy')
back_image = np.load('back_image.npy')

#### 1. Perspective Camera Calibration

Deal with 11 DFs camera matrix with 12 parameters in total, solved through constraint minimization problem with SVD.

#### 2. Affine Camera Calibration

Deal with 8 DFs camera matrix with 8 parameters in total, solved with LSE on linear system.

In [3]:
# world coordinates: (M,3), image_coordinates: (M,2)
def calibrate_perspective_camera(world_coordinates, image_coordinates):
    wc,ic = world_coordinates, image_coordinates
    wc = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1.) # (M,4)
    pstack = np.repeat(wc, repeats=2, axis=0) # (2M,4)
    # scale even & odd rows with different values
    v1 = np.array([1,0])
    tmp1 = np.tile(v1,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    v2 = np.array([0,1])
    tmp2 = np.tile(v2,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    pix = ic.reshape(-1,1) # (2M,1)    
    tmp3 = - pix * pstack # (2M,4)
    P = np.concatenate((tmp1,tmp2,tmp3),axis=1) # (2M,12)
    # optimization problem solved with Singular Value Decomposition
    u,s,vh = np.linalg.svd(P)
    # take last column of V, or the last row of V.T: (12,)
    sol = vh[-1]
    # construct camera matrix M: (3,4)
    M = sol.reshape(3,4)
    return M

# affine camera model: Standard Least Squares
def calibrate_affine_camera(world_coordinates, image_coordinates):
    wc,ic = world_coordinates, image_coordinates
    wc = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1.) # (M,4)
    pstack = np.repeat(wc, repeats=2, axis=0) # (2M,4)
    # scale even & odd rows with different values
    v1 = np.array([1,0])
    tmp1 = np.tile(v1,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    v2 = np.array([0,1])
    tmp2 = np.tile(v2,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    P = np.concatenate((tmp1,tmp2),axis=1) # (2M,8)
    b = image_coordinates.reshape(-1,1) # (2M,1)
    m_ls = np.linalg.inv(P.T@P)@(P.T)@b # (8,1)
    # construct affine camera matrix
    M12 = m_ls.reshape(2,4) # (2,4)
    M3 = np.array([[0.,0.,0.,1.]]) # (1,4)
    M = np.concatenate((M12,M3),axis=0)
    return M

To generalize the usage, specify world_coordinates (-1,3) and image_coordinates (-1,2) for application of above function.

In [4]:
# world coordinate for first imageL (N,3)
wc1 = np.pad(real_XY,[(0,0),(0,1)],mode='constant',constant_values=0.)
# world coordinate for second image: (N,3)
wc2 = np.pad(real_XY,[(0,0),(0,1)],mode='constant',constant_values=150.)
# world coordinates (3D) and image coordinate (2D) construction
world_coordinates = np.concatenate((wc1,wc2),axis=0) # (2N,3)
image_coordinates = np.concatenate((front_image,back_image)) # (2N,2)

# fit the perspective camera model
M0 = calibrate_perspective_camera(world_coordinates, image_coordinates)
M1 = calibrate_affine_camera(world_coordinates, image_coordinates)
print('Calibrated Perspective Camera model is:\n',M0)
print('\n Calibrated Affine Camera model is:\n',M1)

Calibrated Perspective Camera model is:
 [[ 3.86081985e-03 -1.14839115e-04  8.75272791e-04  9.46068598e-01]
 [ 3.42814033e-04  3.92515324e-03 -7.51681648e-04  3.23835234e-01]
 [-6.97426063e-08  8.23266292e-08 -1.33752672e-08  7.29214554e-03]]

 Calibrated Affine Camera model is:
 [[ 5.31276507e-01 -1.80886074e-02  1.20509667e-01  1.29720641e+02]
 [ 4.84975447e-02  5.36366401e-01 -1.02675222e-01  4.43879607e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


#### 3. RMS Error

In [5]:
def rms_error(world_coordinates, image_coordinates, camera_matrx):
    wc,ic,M = world_coordinates, image_coordinates, camera_matrx
    # obtain predicted image coordinate
    wc_proj = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1) # (M,4)
    pred_ic_proj = (M @ wc_proj[:,:,np.newaxis])[:,:,0] # (M,3)
    pred_ic = pred_ic_proj[:,:2]/pred_ic_proj[:,2:] # (M,2)
    # compute RMS
    sq_diff = np.sum((pred_ic - ic)**2,axis=1)
    rms = np.sqrt(np.mean(sq_diff))
    return rms

print('RMS for calibrated perspective camera model is:',rms_error(world_coordinates, image_coordinates, M0))
print('RMS for calibrated affine camera model is:',rms_error(world_coordinates, image_coordinates, M1))

RMS for calibrated perspective camera model is: 0.9916933372299293
RMS for calibrated affine camera model is: 0.9938304832798437


#### 4. Compute Vanishing points

In [6]:
# input four points: pts[0]-pts[1] form one parallel line, pts[2]-pts[3] another
def compute_vanishing_point(pts):
    a1 = (pts[1] - pts[0])[::-1] * np.array([1,-1])
    a2 = (pts[3] - pts[2])[::-1] * np.array([1,-1])
    c1 = np.sum(pts[0]*a1)
    c2 = np.sum(pts[2]*a2)
    v = np.linalg.solve(np.stack((a1,a2)), np.stack((c1,c2)))
    return v

# for derivation process see deri.ipynb
def compute_K_from_vanishing_points(vanishing_points):
    v1,v2,v3 = vanishing_points
    A1 = np.array([v1[0]*v2[0]+v1[1]*v2[1],v1[0]+v2[0],v1[1]+v2[1],1])
    A2 = np.array([v1[0]*v3[0]+v1[1]*v3[1],v1[0]+v3[0],v1[1]+v3[1],1])
    A3 = np.array([v2[0]*v3[0]+v2[1]*v3[1],v2[0]+v3[0],v2[1]+v3[1],1])
    A = np.vstack((A1,A2,A3))
    # solve for w', the wrongly scaled solution
    u,s,vt = np.linalg.svd(A)
    w = vt[-1]
    # construct omega from w values
    omega = np.array([[w[0],0,w[1]],
                      [0,w[0],w[2]],
                      [w[1],w[2],w[3]]])
    # Cholesky decomposition to get wrongly scaled intrinsic matrix K
    KT_inv = np.linalg.cholesky(omega)
    K = np.linalg.inv(KT_inv.T)
    # normalized K to the correctly scaled, thanks to the fact that K[2,2]=1
    K /= K[2,2]
    return K

Derivation process for $(e)$ question:

1. To compute the angle between two planes (in world 3D coordinate), we need their normal vector's direction (only ratio matters not scale).
2. Given 2 pairs of two vanishing points, which are points on image plane (2D) which is projection of point at infinity for two different parallel lines on each plane.
3. On plane 1, we have two different point at infinity $(d_{1x},d_{1y},d_{1z},0)$, $(d_{2x},d_{2y},d_{2z},0)$ which implies that the first point are interception of all paralle lines having form $\lambda(d_{1x},d_{1y},d_{1z})+(x_{0},y_{0}z_{0},1)$ for $(x_{0},y_{0},z_{0}) \in \mathbb{R}^{3}$ and also on plane 1 ,$\lambda \in \mathbb{R}$, and second point is simply the ideal point in $\mathbb{P}^{3}$ which correspond to parallel line with direction $(d_{2x},d_{2y},d_{2z})$
4. Since the normal vector for plane 1 is orthogonal to all lines on plane 1, we have $n^{T}d_{1}=n^{T}d_{2}=0$, we can also write $n = d_{1}\times d_{2}$ where we use the cross product, note again that scale doens't matter for $n$, only the ratio or 'direction' does.
5. Now, we have $v=Kd$, therefore by standard property of cross product we have that

$$
n = d_{1} \times d_{2} = K^{-1}v_{1} \times K^{-1}v_{2} = \det(K)^{-1}K^{T}(v_{1}\times v_{2}) \propto K^{T}(v_{1}\times v_{2})
$$

6. Say we have normal vectors for the two plane, $n_{1}\propto K^{T}(v_{1}\times v_{2}), n_{2}\propto K^{T}(v_{3}\times v_{4})$. Then angles of two plane are the same as (the smaller) angle of two normal vectors.
$$
\cos(\angle plane_1 plane_2) = \cos(\angle n_{1}n_{2}) = \frac{n_{1}^{T}n_{2}}{\|n_{1}\| \|n_{2}\|}
$$
where we note that 
$$
n_{1}^{T}n_{2} = det(K)^{-2} (v_{1}\times v_{2})^{T} KK^{T} (v_{3}\times v_{4})
$$
the scale of $n_{1},n_{2}$ doesn't matter because the scaling factor get cancelled out during computation of $\cos(\angle(n_{1}n_{2}))$
7. Since $\omega = (KK^{T})^{-1}$, we have 
$$
n_{1}^{T}n_{2} = (v_{1}\times v_{2})^{T} \omega^{-1} (v_{3}\times v_{4})
$$

In [58]:
def compute_angle_between_planes(vanishing_pair1, vanishing_pair2, K):
    # pad to trun vanihsing points from R2 into P2 projective coordinates
    v1 = np.pad(vanishing_pair1[0],(0,1),constant_values=1.)
    v2 = np.pad(vanishing_pair1[1],(0,1),constant_values=1.)
    v3 = np.pad(vanishing_pair2[0],(0,1),constant_values=1.)
    v4 = np.pad(vanishing_pair2[1],(0,1),constant_values=1.)
    # compute cross product for multiple reusing
    l1 = np.cross(v1,v2)
    l2 = np.cross(v3,v4)
    l1 /= np.linalg.norm(l1)
    l2 /= np.linalg.norm(l2)
    # compute omega inverse 
    inv_omega = K@(K.T)
    # compute cos(theta)
    cos_theta = (l1.T@inv_omega@l2)/(np.sqrt(l1.T@inv_omega@l1)*np.sqrt(l2.T@inv_omega@l2))
    # retrieve the angle from its cosine value
    theta = (np.arccos(cos_theta)/np.pi) * 180
    
    return theta

Derivation for (f).
1. under real world coordinate, line with direction $(d_x,d_y,d_z)$ got translated into 1st camera coordinate through 
$$
d_{1} = [R_{1},T_{1}]d
$$ 
translated into 2nd camera coordinate through 
$$
d_{2} = [RR_{1},T_{1}]d = Rd_{1}
$$ where we would like to solve for the rotation matrix $R$. 
2. There are 3 DFs related to $R$, thus we use 3 different corresponding vanishing points (correspond to 3 lines/directions in 3D world/camera coordinates for the two images taken with different camera angles), then we do simple inverse and solve for $R$.
$$
R[d_{1}^{1},d_{1}^{2},d_{1}^{3}] = [d_{2}^{1},d_{2}^{2},d_{2}^{3}] \\
R = [d_{2}^{1},d_{2}^{2},d_{2}^{3}] [d_{1}^{1},d_{1}^{2},d_{1}^{3}]^{-1} \\
R = D_{2} D_{1}^{-1}
$$
3. Note that we have vanishing points $p_{1},p_{2}$, satisfying
$$
p_{1}=Kd_{1}, d_{1}=K^{-1}p_{1}
$$
where $p_{1}$ need to be in $\mathbb{P}^{2}$, projective coordinate (3-dimensional)
4. We need to normalize $d_{i}$ such that $\|d_{i}\|=1$, since we don't have constraint on $v_{i}$'s scale due to nature of projective coordinates and $\det(K)$ is not constrained, we need to normalize on $d_{i}$ to ensure our $R$ is correct.

In [153]:
def compute_rotation_matrix_between_cameras(vanishing_pts1, vanishing_pts2, K):
    v1,v2,v3 = vanishing_pts1
    v1b,v2b,v3b = vanishing_pts2
    V1 = np.pad(np.vstack((v1,v2,v3)),[(0,0),(0,1)],constant_values=1.0).T
    V2 = np.pad(np.vstack((v1b,v2b,v3b)),[(0,0),(0,1)],constant_values=1.0).T
    inv_K = np.linalg.inv(K)
    D1 = inv_K @ V1
    D2 = inv_K @ V2
    # normalize on d=K^{-1}v seperately such that ||d||=1
    D1 /= np.linalg.norm(D1,axis=0)
    D2 /= np.linalg.norm(D2,axis=0)
    # compute R with matrix inversion
    R = D2 @ np.linalg.inv(D1)
    return R

In [161]:
from utils import mat2euler

# Part A: Compute vanishing points.
v1 = compute_vanishing_point(np.array(
        [[674, 1826], [2456, 1060], [1094, 1340], [1774, 1086]]))
v2 = compute_vanishing_point(np.array(
        [[674, 1826], [126, 1056], [2456, 1060], [1940, 866]]))
v3 = compute_vanishing_point(np.array(
        [[1094, 1340], [1080, 598], [1774, 1086], [1840, 478]]))

v1b = compute_vanishing_point(np.array(
        [[314, 1912], [2060, 1040], [750, 1378], [1438, 1094]]))
v2b = compute_vanishing_point(np.array(
        [[314, 1912], [36, 1578], [2060, 1040], [1598, 882]]))
v3b = compute_vanishing_point(np.array(
        [[750, 1378], [714, 614], [1438, 1094], [1474, 494]]))

# Part B: Compute the camera matrix.
vanishing_points = [v1, v2, v3]
K_ours = compute_K_from_vanishing_points(vanishing_points)
#print "Intrinsic Matrix:\n", K_ours

K_actual = np.array([[2448.0, 0, 1253.0], [0, 2438.0, 986.0], [0, 0, 1.0]])
#print
#print "Actual Matrix:\n", K_actual

# Part D: Estimate the angle between the box and floor.
floor_vanishing1 = v1
floor_vanishing2 = v2
box_vanishing1 = v3
box_vanishing2 = compute_vanishing_point(np.array(
        [[1094, 1340], [1774, 1086], [1080, 598], [1840, 478]]))
angle = compute_angle_between_planes(
            [floor_vanishing1, floor_vanishing2],
            [box_vanishing1, box_vanishing2], K_actual)

# Part E: Compute the rotation matrix between the two cameras.
rotation_matrix = compute_rotation_matrix_between_cameras(
        np.array([v1, v2, v3]), np.array([v1b, v2b, v3b]), K_actual)
z, y, x = mat2euler(rotation_matrix)
x_angle = x * 180 / math.pi
y_angle = y * 180 / math.pi
z_angle = z * 180 / math.pi